# LAB: Workflow de Datos. Limpieza y sumarización de datos para el Desafío 3.

## Introducción

La idea de este lab es comenzar a trabajar con los datos del Desafío 3. Vamos a tratar de comenzar con el proceso de limpieza, sumarización y análisis exploratorio del dataset de Properatti. Como recordarán, el objetivo final es el desarrollo de un tasador automático a ser aplicado a las próximas propiedades que sean comercializadas por la empresa mediante un modelo de regresión. Como verán, el dataset está sumamente "sucio" y require un intenso proceos de limpieza.

El dataset contiene información referida al primer semestre de 2017 de las propiedades comercializables por la empresa:

* Fecha de creación
* Tipo de la propiedad (house, apartment, ph)
* Operación del aviso (sell, rent)
* Nombre del lugar
* Nombre del lugar + nombre de sus ‘padres’
* ID de geonames del lugar (si está disponible)
* Latitud,Longitud
* Precio original del aviso
* Moneda original del aviso (ARS, USD)
* Precio del aviso en moneda local (ARS)
* Precio aproximado en USD
* Superficie en m²
* Superficie cubierta en m²
* Precio en USD/m²
* Precio por m²
* N° de piso, si corresponde
* Ambientes
* URL en Properati
* Descripción
* Título
* URL de un thumbnail de la primer foto


Una vez terminada la limpieza, aquellos que se animen pueden empezar a probar algunos modelos simples de regresión.


## Objetivos de aprendizaje

* Practicar técnicas de limpieza de datos de tipo texto
* Practicar conversiones de tipo de dato
* Practicar rellenar datos faltantes con 0s o promedios
* Practicar técnicas para trabajo con datos categóricos
* Transformar datos a información útil
* Realizar un primer análisis exploratorio
* BONUS: realizar un primer modelo de regresión

In [1]:
% matplotlib inline
import matplotlib.pyplot as plt
import datetime
import numpy as np
import pandas as pd

In [3]:
# Cargar la data
df = pd.read_csv('../data/properatti.csv')  # DATOS DE PROPIEDADES EN VENTA PROPERATI
print(df.columns)

Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')


In [4]:
df.head()

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...


# Limpiar el dataset
Practiquemos nuestras capacidades para realizar limpieza de datos con el dataset de ventas de propiedades de Properatti. Si no recuerdan cómo hacer alguna de estas tareas, busquen en Internet o en clases anteriores de manipulación de datos.

Buscamos que completen las siguientes tareas:
* Quitar columnas redundantes
* Convertir las variables categóricas numéricas a integer
* Lidiar con los valores perdidos y/o erróneos en algunas variables clave: 

    + en este punto podría ser interesante realizar algo ligeramente más complejo que un simple rellenado en función de medias. Podría eventualmente imputar en función de medias condicionadas.
        * por ejemplo, los campos `lat` y `long` tienen, también, muchos casos perdidos. Quizás una estrategia posible sería tratar de imputar las coordenadas con la media de los casos en el mismo `place_name` -barrio-.
    + también podrían evaluar la posibilidad de extraer alguna información para completar los datos perdidos del campo "description" que contiene texto del aviso. 
        * por ejemplo, el campo `rooms` tiene una altísima cantidad de missing. Podrían intentar usando alguna expresión regex extraer información 

In [5]:
dfFiltrado = df[(df['state_name'] == 'Capital Federal') & ((df['place_name'] == 'Barrio Norte') | (df['place_name'] == 'Recoleta') )]
dfFiltrado.columns = ['indice', 'operacion', 'tipo_propiedad', 'barrio',
       'resumen_zona', 'pais', 'provincia', 'geonames_id',
       'latitud_longitud', 'latitud', 'longitud', 'precio', 'moneda',
       'precio_pesos', 'precio_usd', 'superficie_m2',
       'superficie_cubierta', 'precio_usd_m2', 'precio_m2', 'piso',
       'habitaciones', 'expensas', 'properati_url', 'descripcion', 'titulo',
       'imagen']

In [6]:
dfFiltrado = dfFiltrado[dfFiltrado['tipo_propiedad'] == 'apartment']

In [7]:
dfFiltrado = dfFiltrado[['barrio',
       'geonames_id',
       'latitud', 'longitud', 'precio', 'moneda',
       'precio_pesos', 'precio_usd', 'superficie_m2',
       'superficie_cubierta', 'precio_usd_m2', 'precio_m2', 'piso',
       'habitaciones', 'expensas', 'properati_url', 'descripcion', 'titulo',
       'imagen']]

In [8]:
cambio = 17.6445
valoresSinPrecio = dfFiltrado[dfFiltrado['precio_usd'].isnull()]

In [9]:
dfFiltrado.shape

(2500, 19)

In [10]:
dfFiltrado = dfFiltrado[~dfFiltrado['precio_usd'].isnull()]

In [11]:
dfFiltrado = dfFiltrado[ dfFiltrado['precio_m2'] >= dfFiltrado['precio_usd_m2']]

In [12]:
dfFiltrado[dfFiltrado['precio_m2'] < dfFiltrado['precio_usd_m2']]

,barrio,geonames_id,latitud,longitud,precio,moneda,precio_pesos,precio_usd,superficie_m2,superficie_cubierta,precio_usd_m2,precio_m2,piso,habitaciones,expensas,properati_url,descripcion,titulo,imagen


In [13]:
dfFiltrado[ dfFiltrado['precio'] != dfFiltrado['precio_usd']]

,barrio,geonames_id,latitud,longitud,precio,moneda,precio_pesos,precio_usd,superficie_m2,superficie_cubierta,precio_usd_m2,precio_m2,piso,habitaciones,expensas,properati_url,descripcion,titulo,imagen
44035,Recoleta,3429595.0,-34.592608,-58.410879,1568100.0,ARS,1551742.20,87944.81,36.0,31.0,2442.911389,50583.870968,NaN,1.0,NaN,http://www.properati.com.ar/18zkn_venta_depart...,1 AMBIENTE CON BALCÓN - BAÑO COMPLETO - COCINA...,1 Ambiente con Balcón,https://thumbs4.properati.com/9/-jVn9mTvFU9Mh8...
44038,Recoleta,3429595.0,-34.592608,-58.410879,1340900.0,ARS,1326912.27,75202.60,37.0,25.0,2032.502703,53636.000000,NaN,1.0,NaN,http://www.properati.com.ar/18zl0_venta_depart...,1 AMBIENTE CON BALCÓN - BAÑO COMPLETO - COCINA...,1 Ambiente con Balcón,https://thumbs4.properati.com/5/v7dPVfTVh64Di-...
44040,Recoleta,3429595.0,-34.592608,-58.410879,2657600.0,ARS,2629876.90,149047.97,82.0,47.0,1817.658171,56544.680851,NaN,2.0,NaN,http://www.properati.com.ar/18zl4_venta_depart...,2 AMBIENTES CON BALCÓN TERRAZA AL FRENTE - BAÑ...,2 Ambientes con Balcón Terraza al Frente,https://thumbs4.properati.com/4/n278jXgCPlISfa...
44043,Recoleta,3429595.0,-34.592608,-58.410879,2514100.0,ARS,2487873.97,140999.97,53.0,47.0,2660.376792,53491.489362,NaN,2.0,NaN,http://www.properati.com.ar/18zl8_venta_depart...,2 AMBIENTES CON BALCÓN AL FRENTE - BAÑO COMPLE...,2 Ambientes con Balcón al Frente,https://thumbs4.properati.com/4/XXDdfmKL7hOoW7...
44044,Recoleta,3429595.0,-34.592608,-58.410879,3198900.0,ARS,3165530.40,179406.07,95.0,47.0,1888.484947,68061.702128,NaN,2.0,NaN,http://www.properati.com.ar/18zla_venta_depart...,2 AMBIENTES AL FRENTE CON TERRAZA EXCLUSIVA - ...,2 Ambientes al Frente con Terraza Exclusiva,https://thumbs4.properati.com/8/C8LsLcWjzmbQUA...
44045,Recoleta,3429595.0,-34.592608,-58.410879,1845000.0,ARS,1825753.69,103474.38,56.0,26.0,1847.756786,70961.538462,NaN,1.0,NaN,http://www.properati.com.ar/18zlb_venta_depart...,1 AMBIENTE AL CONTRAFRENTE CON TERRAZA EXCLUSI...,1 Ambiente al Contrafrente con Terraza Exclusiva,https://thumbs4.properati.com/8/14iYmvuZ_V3wpL...
44664,Recoleta,3429595.0,-34.601159,-58.398106,1600537.0,ARS,1583840.72,89763.99,60.0,30.0,1496.066500,53351.233333,NaN,1.0,NaN,http://www.properati.com.ar/19112_venta_depart...,1 AMBIENTE CON PATIO AL CONTRAFRENTE - BAÑO CO...,1 Ambiente con Patio al Contrafrente,https://thumbs4.properati.com/3/NsxpshxmG7wacP...
44665,Recoleta,3429595.0,-34.601159,-58.398106,1492026.0,ARS,1476461.76,83678.30,57.0,27.0,1468.040351,55260.222222,NaN,1.0,NaN,http://www.properati.com.ar/19117_venta_depart...,1 AMBIENTE CON PATIO AL CONTRAFRENTE - BAÑO CO...,1 Ambiente con Patio al Contrafrente,https://thumbs4.properati.com/9/EZwlKngJ19NCaI...
44666,Recoleta,3429595.0,-34.601159,-58.398106,2218764.0,ARS,2195618.76,124436.44,51.0,42.0,2439.930196,52827.714286,NaN,2.0,NaN,http://www.properati.com.ar/1911a_venta_depart...,1 AMBIENTE CON PATIO AL CONTRAFRENTE - BAÑO CO...,2 Ambientes con Balcón al Frente,https://thumbs4.properati.com/5/N4649fSUBNMqvN...
44669,Recoleta,3429595.0,-34.601159,-58.398106,2313180.0,ARS,2289049.74,129731.63,51.0,42.0,2543.757451,55075.714286,NaN,2.0,NaN,http://www.properati.com.ar/1911e_venta_depart...,1 AMBIENTE CON PATIO AL CONTRAFRENTE - BAÑO CO...,2 Ambientes con Balcón al Frente,https://thumbs4.properati.com/4/Mp6ComWGKTKJTq...


In [14]:
dfFiltrado = dfFiltrado[['barrio', 'geonames_id', 'latitud', 'longitud', 'precio_usd', 'superficie_m2', 'superficie_cubierta',
       'precio_usd_m2', 'piso', 'habitaciones', 'expensas',
       'properati_url', 'descripcion', 'titulo', 'imagen']]

In [15]:
dfFiltrado['precio_usd_m2_calculado'] = dfFiltrado['precio_usd'] / dfFiltrado['superficie_m2']

In [16]:
(dfFiltrado['precio_usd_m2'].astype(int) == dfFiltrado['precio_usd_m2_calculado'].astype(int)).value_counts()

True    1870
dtype: int64

In [17]:
dfFiltrado.drop('precio_usd_m2_calculado', axis=1, inplace = True)

In [18]:
pd.set_option('display.max_colwidth', 50)
dfFiltrado[dfFiltrado['descripcion'].str.lower().str.contains('al frente')]['descripcion']

200       Los equipos de aire acondicionado no estan inc...
227       Espectacular ubicación, al frente orientación ...
412       2º piso al frente con balcón y grandes ventana...
697       VENTA DEPARTAMENTO 2 AMBIENTES BARRIO NORTEImp...
712       Venta de Departamento 3 AMBIENTES en Barrio No...
1134      Excelente piso de categoría al frente, con  de...
1142      Excelente piso antiguo. Pisos de parquet de ro...
1143      Excelente piso antiguo de estilo racionalista,...
1166      Lindisimo piso, en la mejor zona de Recoleta, ...
1309      Semipiso al frente, living comedor a balcón co...
1371      Edificio de categoría, frente al Hotel Alvear,...
1533      CODIGO: 572-2515 ubicado en: Pacheco de Melo 2...
4314      Piso único con gran recepción, comedor y escri...
4668      -Departamento  de 88m2 y 82m2 cubiertos, al fr...
5243      CERCA DE PARQUE LAS HERAS Y JARDÍN BOTÁNICO, E...
5440      Muy lindo departamento de 4 ambientes en muy b...
5500      Semi piso muy luminoso y con r

In [19]:
dfFiltrado.shape

(1870, 15)

In [20]:
dfFiltrado = dfFiltrado.drop_duplicates(subset='descripcion')

In [21]:
dfFiltrado.shape

(1648, 15)

In [22]:
pd.set_option('display.max_colwidth', -1)
dfFiltrado['frente'] = dfFiltrado['descripcion'].str.lower().str.contains('al frente').apply(lambda x: 1 if x == True else 0)

In [23]:
dfFiltrado['frente'].value_counts()

0    1172
1    476 
Name: frente, dtype: int64

In [24]:
dfFiltrado['descripcion'].str.lower().str.contains('contrafrente').value_counts()
dfFiltrado['descripcion'].str.lower().str.contains('al frente').value_counts()

dfFiltrado[dfFiltrado['titulo'].str.lower().str.contains('al frente')]['frente']

200       1
6149      1
7392      1
9295      0
9481      1
10074     1
12998     1
13084     1
13324     0
13773     0
14080     0
14812     1
14816     1
15266     1
15991     1
17131     1
18162     1
23182     1
23476     1
24655     1
24658     1
24659     1
24905     0
25270     1
27536     1
27826     1
28202     1
30206     1
30218     1
30454     1
         ..
57059     0
57067     1
58092     1
58099     1
58706     1
58916     1
58917     1
59375     1
59861     0
60361     1
60362     1
61836     1
61934     1
62143     1
72444     0
72902     1
86211     0
92555     1
114028    1
114034    1
114060    0
114411    1
114445    0
116898    1
119282    1
119283    0
119284    0
119285    0
119329    1
119413    1
Name: frente, Length: 87, dtype: int64

In [25]:
dfFiltrado['titulo'].str.lower().str.contains('al frente') | dfFiltrado['descripcion'].str.lower().str.contains('al frente')

200       True 
227       True 
255       False
262       False
412       True 
422       False
636       False
641       False
643       False
645       False
697       True 
712       True 
840       False
1093      False
1134      True 
1139      False
1141      False
1142      True 
1143      True 
1162      False
1166      True 
1170      False
1173      False
1175      False
1274      False
1278      False
1309      True 
1341      False
1366      False
1369      False
          ...  
119284    True 
119285    True 
119288    False
119296    False
119299    False
119324    False
119325    True 
119326    True 
119328    True 
119329    True 
119340    False
119362    False
119375    False
119380    True 
119401    False
119408    False
119413    True 
119545    False
119798    False
119806    True 
119998    False
120068    False
120074    False
120217    True 
120303    True 
120935    False
121121    False
121131    True 
121153    False
121158    True 
Length: 1648, dtype: boo

In [26]:
dfFiltrado['frente'] = (dfFiltrado['titulo'].str.lower().str.contains('al frente') | dfFiltrado['descripcion'].str.lower().str.contains('al frente')).apply(lambda x: 1 if x == True else 0)
dfFiltrado[dfFiltrado['titulo'].str.lower().str.contains('al frente')]['frente']

200       1
6149      1
7392      1
9295      1
9481      1
10074     1
12998     1
13084     1
13324     1
13773     1
14080     1
14812     1
14816     1
15266     1
15991     1
17131     1
18162     1
23182     1
23476     1
24655     1
24658     1
24659     1
24905     1
25270     1
27536     1
27826     1
28202     1
30206     1
30218     1
30454     1
         ..
57059     1
57067     1
58092     1
58099     1
58706     1
58916     1
58917     1
59375     1
59861     1
60361     1
60362     1
61836     1
61934     1
62143     1
72444     1
72902     1
86211     1
92555     1
114028    1
114034    1
114060    1
114411    1
114445    1
116898    1
119282    1
119283    1
119284    1
119285    1
119329    1
119413    1
Name: frente, Length: 87, dtype: int64

In [27]:
dfFiltrado['frente'].value_counts()

0    1155
1    493 
Name: frente, dtype: int64

In [28]:
dfFiltrado[dfFiltrado['descripcion'].str.lower().str.contains(' frente')]['descripcion']

200       Los equipos de aire acondicionado no estan incluidos en el precio de venta.Semipiso. Living. Comedor. 4 dormitorio(s) 1 en suite. 2 baño(s) 1 toilette(s). Entre medianeras. Al frente. S/av. principal.  Antiguo.  Luminoso.  En muy buen estado. Orientación Suroeste. Antigüedad 80 años.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [29]:
dfFiltrado.shape

(1648, 16)

In [30]:
dfFiltrado.columns

Index(['barrio', 'geonames_id', 'latitud', 'longitud', 'precio_usd',
       'superficie_m2', 'superficie_cubierta', 'precio_usd_m2', 'piso',
       'habitaciones', 'expensas', 'properati_url', 'descripcion', 'titulo',
       'imagen', 'frente'],
      dtype='object')

In [31]:
dfFiltrado.drop_duplicates(subset=['barrio', 'precio_usd',
       'superficie_m2', 'superficie_cubierta', 'precio_usd_m2', 'piso',
       'habitaciones', 'frente']).shape

(1601, 16)

In [32]:
dfFiltrado = dfFiltrado.drop_duplicates(subset=['barrio', 'precio_usd',
       'superficie_m2', 'superficie_cubierta', 'precio_usd_m2', 'piso',
       'habitaciones', 'frente'])

In [33]:
dfFiltrado['luminoso'] = (dfFiltrado['titulo'].str.lower().str.contains('luminoso') | dfFiltrado['descripcion'].str.lower().str.contains('luminoso')).apply(lambda x: 1 if x == True else 0)

In [34]:
pd.set_option('display.max_colwidth', 10)
dfFiltrado.shape

(1601, 17)

# Filtrar los Datos

En general, los mdoelos de precios suelen tener un carácter local. Es por eso que deberán elegir alguna zona o provincia para trabajar y estimar el modelo en base a dicha selección.

In [35]:
# Filtrar la zona elegida para estimar el modelo



# Análisis exploratorio

Como estamos intentando predecir precios por metro cuadrado es importante realizar una primera vista y exploración de los datos.

In [36]:
# Calculo de correlacion entre features


In [ ]:
# Analisis exploratorio, graficos...


¿En qué barrios los precios por $m^2$ son más altos? 

¿En qué barrios hay una mayor cantidad de viviendas para su venta?

¿ En qué barrios hay una mayor dispersión de precios?

¿Qué tipo de propiedad presenta los mayores precios? ¿Cuál las mayores dispersiones?

Continua realizando un análisis exploratorio del dataset...

## BONUS: Empezando con regresiones lineales...

Si se atreven, podrían empezar a probar con algunos modelos de regresión no demasiado sofisticados. Seleccionen (o construyan) su variable dependiente ($Y$) y comiencen probando con pocas variables y modelos sencillos.

Usen el estimador `LinearRegression` de submódulo `sklearn.linear_model`.

Recuerden los pasos para comenzar a trabajar con estimadores en Scikit-Learn.
    
1. Elegir una clase de modelo importando la clase de estimador apropiado de Scikit-Learn.
2. Seleccionar los hiperparámetros del modelo instanciando la clase con los valores deseados.
3. Preparar los datos en una matriz de features y un array target, como vimos previamente.
4. Ajustar el modelo a los datos invocando el método fit() de la instancia del modelo.
5. Aplicar el modelo a nuevos datos:
    * Para aprendizaje supervisado, frecuentemente predecimos labels para datos nuevos usando el método predict()
    * Para aprendizaje no supervisado, frecuentemente transformamos o inferimos propiedades de los datos usando los métodos transform() o predict()
    
¿Qué pueden decir del ajuste y la capacidad predictiva de sus modelos?

In [ ]:
from sklearn.linear_model import LinearRegression